# **DATA LOADING AND NORMALIZATION**

In [18]:
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import torch
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from sklearn.metrics import precision_score, recall_score
from torchsummary import summary
import math

In [19]:
labels = ['PNEUMONIA', 'NORMAL']
img_size = 244

def get_training_data(data_dir):
    data = []
    
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        
        for img in os.listdir(path):
            try:
                # Load and resize the image
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size))  # Resize the image
                
                # Add the image and label as a pair
                data.append([resized_arr, class_num])
            except Exception as e:
                print(f"Error loading image {img}: {e}")
    
    # Convert the list to a NumPy array
    data = np.array(data, dtype=object)  # Use dtype=object to allow image-label pairing
    return data

# Load the data
train_data = get_training_data('data/chest_xray/train')
test_data = get_training_data('data/chest_xray/test')
val_data = get_training_data('data/chest_xray/val')

## **NORMALIZE DATA**

In [20]:
#Function to normalize the images
def normalize_images(data):
    images = []
    labels = []

    for img, label in data:
        # Normalization: each pixel is divided by 255
        normalized_img = img / 255.0
        images.append(normalized_img)
        labels.append(label)

#Convert the images and labels into separate arrays
    images = np.array(images)
    labels = np.array(labels)

    return images, labels


In [21]:
#Normalize the images in the training dataset
train_images, train_labels = normalize_images(train_data)
val_images, val_labels = normalize_images(val_data)
test_images, test_labels = normalize_images(test_data)

#Shuffle the training and validation data
train_images, train_labels = shuffle(train_images, train_labels, random_state=42)
val_images, val_labels = shuffle(val_images, val_labels, random_state=42)

#Check the shape and an example of the normalized and shuffled data
print(f"Shape of normalized and shuffled train images: {train_images.shape}")
print(f"Shape of normalized and shuffled validation images: {val_images.shape}")


Shape of normalized and shuffled train images: (6214, 244, 244)
Shape of normalized and shuffled validation images: (1046, 244, 244)


In [22]:
# # Separate the images and the labels
# train_images = np.array([x[0] for x in train_data])  # Extract only the images
# train_labels = np.array([x[1] for x in train_data])  # Extract only the labels

# # Display the first image
# plt.figure(figsize=(5, 5))
# plt.imshow(train_images[0])  # The first image
# plt.title('Pneumonia' if train_labels[0] == 0 else 'Normal')  # Set the title based on the label
# plt.axis('off')  # To hide the axes
# plt.show()

# # Display the last image
# plt.figure(figsize=(5, 5))
# plt.imshow(train_images[-1])  # The last image
# plt.title('Pneumonia' if train_labels[-1] == 0 else 'Normal')  # Set the title based on the label
# plt.axis('off')  # To hide the axes
# plt.show()

# **KAN REAL**

In [41]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from sklearn.metrics import precision_score, recall_score
from torchsummary import summary
import math
import torchvision.transforms as transforms
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader, TensorDataset
from sklearn.utils import shuffle  # Assuming 'shuffle' is needed for shuffling data
from kan import * #Here I used a original KAN implentation from https://github.com/KindXiaoming/pykan/blob/master/hellokan.ipynb



# Use PCA to reduce dimensionality to 2D
def apply_pca(images, n_components=2):
    flat_images = images.reshape(images.shape[0], -1)
    pca = PCA(n_components=n_components)
    transformed_images = pca.fit_transform(flat_images)
    return transformed_images

# Dummy data creation for sample
train_images = np.random.random((100, 28, 28))
val_images = np.random.random((20, 28, 28))
test_images = np.random.random((20, 28, 28))
train_labels = np.random.randint(0, 2, 100)
val_labels = np.random.randint(0, 2, 20)

# Apply PCA to get 2D vectors
train_images_pca = apply_pca(train_images)
val_images_pca = apply_pca(val_images)
test_images_pca = apply_pca(test_images)

# Convert to PyTorch tensors
train_tensors = torch.tensor(train_images_pca, dtype=torch.float32)
train_labels_tensors = torch.tensor(train_labels, dtype=torch.long)
val_tensors = torch.tensor(val_images_pca, dtype=torch.float32)
val_labels_tensors = torch.tensor(val_labels, dtype=torch.long)

train_dataset = TensorDataset(train_tensors, train_labels_tensors)
val_dataset = TensorDataset(val_tensors, val_labels_tensors)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

class KAN_Net(nn.Module):
    def __init__(self, input_dim, hidden_dim=[128, 64], output_dim=2, polynomial_order=3, activation=nn.ReLU):
        super(KAN_Net, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.polynomial_order = polynomial_order
        self.activation = activation()

        poly_feature_size = (self.polynomial_order + 1) * self.input_dim

        # Define layers
        self.base_layers = nn.ModuleList()
        in_dim = self.input_dim
        for dim in hidden_dim:
            self.base_layers.append(nn.Linear(in_dim, dim))
            in_dim = dim

        self.poly_weights = nn.ParameterList([
            nn.Parameter(torch.randn(dim, poly_feature_size)) for dim in hidden_dim
        ])

        self.output_layer = nn.Linear(in_dim, output_dim)

        # Initialize weights
        for layer in self.base_layers:
            nn.init.kaiming_uniform_(layer.weight, nonlinearity='linear')
        for weight in self.poly_weights:
            nn.init.kaiming_uniform_(weight, nonlinearity='linear')

    @staticmethod
    def compute_legendre_polynomials(x, order):
        P0 = torch.ones_like(x)
        P1 = x
        legendre_polys = [P0, P1]

        for n in range(1, order):
            Pn = ((2.0 * n + 1.0) * x * legendre_polys[-1] - n * legendre_polys[-2]) / (n + 1.0)
            legendre_polys.append(Pn)
        
        return torch.cat(legendre_polys, dim=-1)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Update this line

        for layer, poly_weight in zip(self.base_layers, self.poly_weights):
            base_output = self.activation(layer(x))

            # Normalize x for Legendre polynomials
            x_min = x.min(dim=1, keepdim=True)[0]
            x_max = x.max(dim=1, keepdim=True)[0]
            x_normalized = 2 * (x - x_min) / (x_max - x_min) - 1
            legendre_basis = self.compute_legendre_polynomials(x_normalized.unsqueeze(-1), self.polynomial_order)

            legendre_basis = legendre_basis.view(x.size(0), -1)  # Flatten the Legendre basis

            if legendre_basis.shape[1] != poly_weight.shape[1]:
                raise RuntimeError(
                    f"Legendre basis dimension {legendre_basis.shape[1]} does not match poly_weight dimension {poly_weight.shape[1]}"
                )

            poly_output = torch.matmul(legendre_basis, poly_weight.T)
            x = self.activation(base_output + poly_output)

        x = self.output_layer(x)
        return x  # Return logits directly, no softmax

# Setup and model initialization
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def initialize_kan_net(width, seed, device):
    torch.manual_seed(seed)
    input_dim = width[0]
    hidden_dim = width[1:]
    model = KAN_Net(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=2)
    model.to(device)
    return model

model = initialize_kan_net(width=[2, 5], seed=42, device=device)

# Print a model summary
summary(model, (2,))

# AdamW optimizer
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
def calculate_precision_recall(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, preds = torch.max(output, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(target.cpu().numpy())

    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')

    return precision, recall

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 5]              15
              ReLU-2                    [-1, 5]               0
              ReLU-3                    [-1, 5]               0
            Linear-4                    [-1, 2]              12
Total params: 27
Trainable params: 27
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [48]:
python
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score

# Apply the transformation to training and validation images
train_images_tensor = torch.tensor(train_images_pca)
val_images_tensor = torch.tensor(val_images_pca)

# The tensors are already in the shape (N, 2) after PCA

train_labels_tensor = torch.tensor(train_labels, dtype=torch.long) 
val_labels_tensor = torch.tensor(val_labels, dtype=torch.long) 

# Create the dataset and DataLoader
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
val_dataset = TensorDataset(val_images_tensor, val_labels_tensor)

# Define the batch size
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Model and device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the loss function and optimizer 
criterion = nn.CrossEntropyLoss()  # For multi-class or binary classification
optimizer = optim.AdamW(model.parameters(), lr=1e-4)  

# Now the data is ready for training and validation

# Function to calculate precision and recall
def calculate_precision_recall(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(torch.float32).to(device), target.to(torch.long).to(device)
            output = model(data)
            _, preds = torch.max(output, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(target.cpu().numpy())
    
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    
    return precision, recall

# Training function with Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100, patience=10):
    best_val_loss = float("inf")
    epochs_without_improvement = 0  # Count the number of epochs without improvement

    for epoch in range(num_epochs):
        model.train()  # Training mode
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(torch.float32).to(device), labels.to(torch.long).to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Compute the loss
            loss = criterion(outputs, labels)

            # Backpropagation
            loss.backward()
            optimizer.step()

            # Compute metrics
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels).item()
            total_preds += labels.size(0)

        # Calculate average loss and accuracy
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = correct_preds / total_preds

        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}")

        # Validation phase
        model.eval()  # Evaluation mode
        val_loss = 0.0
        val_correct_preds = 0
        val_total_preds = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(torch.float32).to(device), labels.to(torch.long).to(device)

                # Forward pass
                outputs = model(inputs)

                # Compute the loss
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)

                # Compute metrics
                _, preds = torch.max(outputs, 1)
                val_correct_preds += torch.sum(preds == labels).item()
                val_total_preds += labels.size(0)

        # Calculate validation loss and accuracy
        val_loss /= len(val_loader.dataset)
        val_accuracy = val_correct_preds / val_total_preds

        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

        # Calculate precision and recall on validation
        val_precision, val_recall = calculate_precision_recall(model, val_loader, device)
        print(f"Validation Precision: {val_precision:.4f}, Validation Recall: {val_recall:.4f}")

        # Early Stopping: stop if no improvement
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
            # Save the model with the best validation loss
            torch.save(model.state_dict(), "best_model_v3.pth")
            print("Model saved!")
        else:
            epochs_without_improvement += 1
            print(f"Early Stopping Counter: {epochs_without_improvement}/{patience}")

        # If the number of epochs without improvement exceeds patience, stop
        if epochs_without_improvement >= patience:
            print("Early stopping triggered. Stopping training.")
            break

# Start training
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100, patience=10)

Epoch 1/100
Train Loss: 0.6795, Train Accuracy: 0.5500
Validation Loss: 0.9571, Validation Accuracy: 0.4500
Validation Precision: 0.5294, Validation Recall: 0.7500
Model saved!
Epoch 2/100
Train Loss: 0.6792, Train Accuracy: 0.5500
Validation Loss: 0.9574, Validation Accuracy: 0.4500
Validation Precision: 0.5294, Validation Recall: 0.7500
Early Stopping Counter: 1/10
Epoch 3/100
Train Loss: 0.6790, Train Accuracy: 0.5500
Validation Loss: 0.9571, Validation Accuracy: 0.4500
Validation Precision: 0.5294, Validation Recall: 0.7500
Early Stopping Counter: 2/10
Epoch 4/100
Train Loss: 0.6789, Train Accuracy: 0.5500
Validation Loss: 0.9572, Validation Accuracy: 0.4500
Validation Precision: 0.5294, Validation Recall: 0.7500
Early Stopping Counter: 3/10
Epoch 5/100
Train Loss: 0.6786, Train Accuracy: 0.5500
Validation Loss: 0.9575, Validation Accuracy: 0.4500
Validation Precision: 0.5294, Validation Recall: 0.7500
Early Stopping Counter: 4/10
Epoch 6/100
Train Loss: 0.6784, Train Accuracy: 0.5

In [49]:
python
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import precision_score, recall_score, f1_score

# Correctly use test_images_pca for testing
test_labels = np.random.randint(0, 2, 20)  # Ensure test_labels is defined
test_images_tensor = torch.tensor(test_images_pca, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)

# Adjust the length of test_labels to match test_images_pca
if len(test_labels_tensor) != len(test_images_tensor):
    test_labels_tensor = test_labels_tensor[:len(test_images_tensor)]

# Create the dataset and DataLoader for the test set
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

def calculate_metrics(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    correct_preds = 0
    total_preds = 0

    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(torch.float32).to(device), target.to(torch.long).to(device)
            output = model(data)
            _, preds = torch.max(output, 1)
            
            correct_preds += torch.sum(preds == target).item()
            total_preds += target.size(0)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(target.cpu().numpy())
    
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')
    accuracy = correct_preds / total_preds

    return precision, recall, f1, accuracy

def test_model(model, test_loader, device):
    precision, recall, f1, accuracy = calculate_metrics(model, test_loader, device)
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1-Score: {f1:.4f}")

model = initialize_kan_net(width=[2, 5], seed=42, device=device)
model.load_state_dict(torch.load("best_model_v3.pth"))

test_model(model, test_loader, device)

Test Accuracy: 0.6000
Test Precision: 0.6667
Test Recall: 0.7692
Test F1-Score: 0.7143


C:\Users\rateb\AppData\Local\Temp\ipykernel_40564\2486844566.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model_v3.pth"))
